In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

# Data augmentations and transformations for Training and Validation
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Flip images randomly
    transforms.RandomRotation(15),      # Rotate images
    transforms.RandomResizedCrop(224),  # Random crop and resize
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Vary brightness, etc.
    transforms.ToTensor(),              # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),      # Resize the images to the input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the Training dataset
train_dataset = datasets.ImageFolder(root='Dataset/Training', transform=train_transform)

# Load the Validation dataset
val_dataset = datasets.ImageFolder(root='Dataset/Validation', transform=val_transform)

# Create DataLoader for Training and Validation
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Print the class indices
print(f"Class indices: {train_dataset.class_to_idx}")



Class indices: {'female': 0, 'male': 1}


In [2]:
model = models.mobilenet_v2(pretrained=True)

model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


# for param in model.features.parameters():
#     param.requires_grad = False
    
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    train_accuracy = 100 * correct / total
    # print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.2f}%')


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total}%')

/home/anishpawar/miniconda3/envs/carla_source/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/anishpawar/miniconda3/envs/carla_source/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.2668, Training Accuracy: 88.48%
Epoch [2/10], Loss: 0.2085, Training Accuracy: 91.69%
Epoch [3/10], Loss: 0.1946, Training Accuracy: 92.30%
Epoch [4/10], Loss: 0.1855, Training Accuracy: 92.73%
Epoch [5/10], Loss: 0.1786, Training Accuracy: 93.00%
Epoch [6/10], Loss: 0.1725, Training Accuracy: 93.44%
Epoch [7/10], Loss: 0.1753, Training Accuracy: 93.24%
Epoch [8/10], Loss: 0.1697, Training Accuracy: 93.26%
Epoch [9/10], Loss: 0.1650, Training Accuracy: 93.56%
Epoch [10/10], Loss: 0.1637, Training Accuracy: 93.63%
Accuracy of the model on the test images: 97.08129453171946%


In [3]:
torch.save(model.state_dict(), 'model_75_xx.pt')

In [5]:
model.load_state_dict(torch.load('model_75_xx.pt', weights_only=True))


<All keys matched successfully>